<a href="https://colab.research.google.com/github/kazzzz65/myRepository/blob/master/implementacja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Załadowanie potrzebnych bibliotek.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import sklearn.metrics as mt
import sklearn.decomposition as dcmp
import sklearn.impute as impute
import sklearn.preprocessing as prprc
from sklearn.utils import resample
import array
import copy 
from collections import Counter
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Załadowanie danych z pliku csv

In [ ]:
test = pd.read_csv("/content/test.csv")
train1 = pd.read_csv("/content/train1.csv")

Funkcje do przygotowania danych i oceny modelu

In [ ]:


def nulls_percentage(data):
    nulls = data.isnull().sum().to_frame()
    data_size = data.count().to_frame().max().iloc[0]
    nulls['Percent missing'] = 100 * nulls / data_size
    nulls = nulls[nulls[0] > 0].sort_values(by = 0, ascending=False)
    return nulls

def product_split(data):
    data['Product_Info_n'] = data['Product_Info_2'].str[1:].astype('int64')
    data['Product_Info_2'] = data['Product_Info_2'].str[0:1]

    data['Product_Info_2'] = data['Product_Info_2'].replace({"A": 1, "B": 2, "C": 3, "D": 4, "E": 5})
    return data

def is_outlier(x, alpha = 0.95):
    if x.dtype == 'float64':
        a = (1 - alpha) / 2
        ql, qu = np.quantile(x[~np.isnan(x)], q=(a, 1 - a))

        outliers = np.logical_or(x < ql, x > qu)
    elif x.dtype == 'int64':
        # Brak outlierów dla całkowitych
        outliers = np.repeat(False, x.size).astype('bool')
    else:
        raise ValueError("nie mozna okreslic outlierow na takim typie danych")

    return outliers

def fix_outliers(x, alpha = 0.95):
    result = x.copy()
    if x.dtype == 'float64':
        a = (1 - alpha) / 2
        x_nans = np.isnan(x)
        ql, qu = np.quantile(x[np.logical_not(x_nans)], q=(a, 1 - a))

        replace_lower = np.nan_to_num(result, ql + 1) < ql
        replace_upper = np.nan_to_num(result, qu - 1) > qu
        result[replace_lower] = ql
        result[replace_upper] = qu
    elif x.dtype != 'int64':
        raise ValueError("nie mozna okreslic outlierow na takim typie danych")

    return result


def prepare_data(data, del_threshold = None, fill_in_func = 'mean',
                      scaling = True,
                      outliers_cols = None,
                      outliers_alpha = None,
                      outliers_method = 'fix',
                      to_numpy = False):

    data_ = copy.deepcopy(data)

    ids = data_['Id']
    data_ = data_.drop('Id', axis = 1)

    target_name = 'Response'
    if target_name in data_.columns:
        target = data_[target_name]
        data_ = data_.drop(target_name, axis = 1)
    else: 
        target = None
        
    target = pd.Series(target)
    target.index = ids
    # Rodzielenie kolumny stringowej i zamiana na liczbowa 
    data_ = product_split(data_)
    
    # Uzupelnianie list z kolumnami do usuniecia i do wypelnienia
    to_be_deleted = []
    nulls = nulls_percentage(data_)
    if del_threshold is not None:
        for i in range(len(nulls)):
            if nulls['Percent missing'][i] >= del_threshold:
                to_be_deleted.append(nulls.index[i])
    
    # Usuniecie kolumn
    data_ = data_.drop(to_be_deleted, axis = 1)
    
    # Usuniecie/naprawa outlierów
    if outliers_alpha is not None:
        assert isinstance(outliers_alpha, float), \
            'outliers_alpha musi być liczbą z przedziału (0, 1)'

        cols = set(data_.columns.values)
        if outliers_cols is not None:
            cols = cols.intersection(set(outliers_cols))

        if outliers_method == 'remove':
            outliers = np.repeat(False, data_.shape[0]).astype('bool')
            for current_var in cols:
                x = data_.loc[:, current_var]
                outliers = np.logical_or(outliers, is_outlier(x, outliers_alpha))

            data_ = data_[~outliers.values]
            target = target[~outliers.values]
            ids = ids[~outliers.values]
        elif outliers_method == 'fix':
            for current_var in cols:
                data_[current_var] = fix_outliers(data_[current_var].values,
                                                  outliers_alpha)
                
    # Wypełnienie nulli
    imputer = impute.SimpleImputer(strategy = fill_in_func)
    data_ = pd.DataFrame(imputer.fit_transform(data_), columns = data_.columns)

    if scaling:
        scaler = prprc.StandardScaler()
        data_ = pd.DataFrame(scaler.fit_transform(data_), columns = data_.columns)
    else:
        scaler = None

   

    if to_numpy:
        data_ = data_.to_numpy()
        target = target.to_numpy()
        
    return (data_, target, to_be_deleted, imputer, scaler, to_numpy)
    
def rearrange_classes(y):
  y = np.array(y)
  a = array.array("i", [])
  for i in y:
    if i > 4:
      a.append(1)
    else:
      a.append(0)
  return a



Przeprowadzenie preprocessingu ze skalowaniem i naprawieniem outlierów.

In [ ]:

data_params = dict(del_threshold=0.05,
                   fill_in_func='mean',
                   outliers_cols=None,
                   outliers_alpha=None,
                   outliers_method='fix',
                   scaling=True,
                   to_numpy=True)

transf_data = prepare_data(train1, **data_params)
x_train, y_train = transf_data[0:2]

In [ ]:
data_params = dict(fill_in_func='mean',
                   outliers_cols=None,
                   outliers_alpha=None,
                   outliers_method='fix',
                   scaling=True,
                   to_numpy=True)

transf_data1 = pd.DataFrame(prepare_data(test, **data_params))
x_test, y_test = transf_data1[0][0:2]

x_test = pd.DataFrame(x_test)[list(pd.DataFrame(x_train).columns)]


/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


Zmiana klasyfikacji na binarną.

In [ ]:
y_re_train = np.array(rearrange_classes(y_train))
y_re_test = np.array(rearrange_classes(y_test))

In [ ]:
SMOTE = SMOTE()

Zbalansowanie zbiorów metodą oversamplingu.


In [ ]:
x_train_SMOTE, y_re_train_SMOTE = SMOTE.fit_resample(x_train, y_re_train)
x_test_SMOTE, y_re_test_SMOTE = SMOTE.fit_resample(x_test, y_re_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
y_re_test_SMOTE.shape

(14812,)

In [ ]:
x_train_SMOTE.shape

(14844, 115)

Model liniowy 

In [ ]:
#%%
model_L = LinearSVC(C=1, dual=False)

model_L.fit(x_train_SMOTE,y_re_train_SMOTE)

LinearSVC(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
pred_L = model_L.predict(x_test_SMOTE)

In [ ]:
print(mt.classification_report(y_re_test_SMOTE, pred_L))

              precision    recall  f1-score   support

           0       0.62      0.53      0.57      7406
           1       0.59      0.68      0.63      7406

    accuracy                           0.60     14812
   macro avg       0.61      0.60      0.60     14812
weighted avg       0.61      0.60      0.60     14812



Model z jądrem RBF.

In [ ]:
model_R = SVC(kernel="rbf",  C=1,  max_iter=10000)

In [ ]:
model_R.fit(x_train_SMOTE, y_re_train_SMOTE)

# Ocena modelu


#%%
pred_R = model_R.predict(x_test_SMOTE)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [ ]:
print(mt.classification_report(y_re_test_SMOTE, pred_R))

              precision    recall  f1-score   support

           0       0.57      0.50      0.53      7406
           1       0.55      0.62      0.59      7406

    accuracy                           0.56     14812
   macro avg       0.56      0.56      0.56     14812
weighted avg       0.56      0.56      0.56     14812



Model wielomianowy

In [ ]:
model_P = SVC(kernel="poly")

In [ ]:
model_P.fit(x_train_SMOTE, y_re_train_SMOTE)

# Ocena modelu


#%%
pred_P = model_P.predict(x_test_SMOTE)

In [ ]:
print(mt.classification_report(y_re_test_SMOTE, pred_P))

              precision    recall  f1-score   support

           0       0.71      0.16      0.27      7406
           1       0.53      0.93      0.67      7406

    accuracy                           0.55     14812
   macro avg       0.62      0.55      0.47     14812
weighted avg       0.62      0.55      0.47     14812



Funkcje do obliczania odległości Mahalanobisa i macierzy K związanej z jądrem k Mahalanobisa.


In [ ]:
def mahalanobis(x=None, data=None, cov=None):

    x_mu = x - np.mean(data)
    inv_covmat = np.linalg.inv(cov)
    left = np.dot(x_mu, inv_covmat)
    mahal = np.dot(left, x_mu.T)
    mahal = mahal.diagonal()
    return mahal.sum()/mahal.size

In [ ]:
def mahalanobisKernelGramMatrixFull(X1, X2, delta=0.5):
    """(Pre)calculates Gram Matrix K"""

    cov = np.cov(X1.values.T)
    m = mahalanobis(X1, X2, cov)
    Q_inv = np.linalg.inv(cov)
    gram_matrix = np.zeros((X1.shape[0], X2.shape[0]))
    X1 = np.array(X1)
    X2 = np.array(X2)
    for i, x1 in enumerate(X1):
        for j, x2 in enumerate(X2):
            x = x1.flatten()
            z = x2.flatten()

            gram_matrix[i, j] = np.exp(- np.matmul(np.matmul((x-z).transpose(), Q_inv), x-z)*delta/m)
    gram_matrix = pd.DataFrame(gram_matrix)
    return gram_matrix

Model z jądrem Mahalanobisa.

In [ ]:
x1 = pd.DataFrame(x_train_SMOTE)
K =mahalanobisKernelGramMatrixFull(x1,x1)

In [ ]:
x2 = pd.DataFrame(x_test_SMOTE)
K_test = mahalanobisKernelGramMatrixFull(x2, x1)

In [ ]:
model = SVC( kernel="precomputed", C=1000, max_iter=9)
model.fit(K, y_re_train_SMOTE)

print(model)

SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='precomputed', max_iter=9, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=9).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [ ]:
pred = model.predict(K_test)
print(mt.classification_report(y_re_test_SMOTE, pred))

              precision    recall  f1-score   support

           0       0.54      0.26      0.35      7406
           1       0.51      0.78      0.62      7406

    accuracy                           0.52     14812
   macro avg       0.52      0.52      0.48     14812
weighted avg       0.52      0.52      0.48     14812



In [ ]:
mt.confusion_matrix(y_re_test_SMOTE, pred)

array([[1909, 5497],
       [1656, 5750]])

Zapis modelu do formatu .sav

In [ ]:
import pickle
filename = '/content/mahal_model.sav'
pickle.dump(model, open(filename, 'wb'))

Odczyt modelu.

In [ ]:
import pickle
filename = '/content/mahal_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
print(loaded_model)

SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='precomputed', max_iter=9, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


Macierz K

In [ ]:
print(K)

          0         1         2      ...     14841     14842     14843
0      1.000000  0.858854  0.512305  ...  0.660348  0.526623  0.720025
1      0.858854  1.000000  0.534114  ...  0.631837  0.504329  0.708315
2      0.512305  0.534114  1.000000  ...  0.442861  0.338122  0.505827
3      0.383409  0.428929  0.283821  ...  0.321604  0.243146  0.352029
4      0.823200  0.767267  0.492098  ...  0.608641  0.448307  0.748232
...         ...       ...       ...  ...       ...       ...       ...
14839  0.757737  0.768140  0.526193  ...  0.660426  0.519013  0.718719
14840  0.495620  0.476501  0.343639  ...  0.404825  0.306390  0.458449
14841  0.660348  0.631837  0.442861  ...  1.000000  0.442927  0.620394
14842  0.526623  0.504329  0.338122  ...  0.442927  1.000000  0.469345
14843  0.720025  0.708315  0.505827  ...  0.620394  0.469345  1.000000

[14844 rows x 14844 columns]


In [ ]:
x_te_SMOTE.shape

(14812, 115)